In [ ]:
try:
    import cv2
except:
    %pip install opencv-python-headless==4.9.0.80

import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import re
import json
import pandas as pd
import datetime
from multiprocessing import Pool, cpu_count
import shutil
import cv2
import shutil
from pathlib import Path

print('import successfull')

### Constants

In [ ]:
BASE_PATH = Path('/home/jovyan/work/videos/cross-label/second_coder')

In [ ]:
video_paths = [
"/home/jovyan/work/output/START_S005_T1_La1/START_S005_T1_La1.MP4",
"/home/jovyan/work/output/START_S005_T1_La2/START_S005_T1_La2.MP4",
"/home/jovyan/work/output/START_S009_T1_La1/START_S009_T1_La1.MP4",
"/home/jovyan/work/output/START_S009_T1_La2/START_S009_T1_La2.MP4",
"/home/jovyan/work/output/START_S012_T1_La1/START_S012_T1_La1.MP4",
"/home/jovyan/work/output/START_S012_T1_La2/START_S012_T1_La2.MP4",
"/home/jovyan/work/output/START_S017_T1_La1/START_S017_T1_La1.MP4",
"/home/jovyan/work/output/START_S017_T1_La2/START_S017_T1_La2.MP4",
"/home/jovyan/work/output/START_S021_T1_La1/START_S021_T1_La1.MP4",
"/home/jovyan/work/output/START_S021_T1_La2/START_S021_T1_La2.MP4",
"/home/jovyan/work/output/START_S026_T1_La1/START_S026_T1_La1.MP4",
"/home/jovyan/work/output/START_S026_T1_La2/START_S026_T1_La2.MP4",
"/home/jovyan/work/output/START_S029_T1_La1/START_S029_T1_La1.MP4",
"/home/jovyan/work/output/START_S029_T1_La2/START_S029_T1_La2.MP4",
"/home/jovyan/work/output/START_S033_T1_La1/START_S033_T1_La1.MP4",
"/home/jovyan/work/output/START_S033_T1_La2/START_S033_T1_La2.MP4",
"/home/jovyan/work/output/START_S035_T1_La1/START_S035_T1_La1.MP4",
"/home/jovyan/work/output/START_S035_T1_La2/START_S035_T1_La2.MP4",
"/home/jovyan/work/output/START_S037_T1_La1/START_S037_T1_La1.MP4",
"/home/jovyan/work/output/START_S037_T1_La2/START_S037_T1_La2.MP4",
"/home/jovyan/work/output/START_S041_T1_La1/START_S041_T1_La1.MP4",
"/home/jovyan/work/output/START_S041_T1_La2/START_S041_T1_La2.MP4",
"/home/jovyan/work/output/START_S042_T1_La1/START_S042_T1_La1.MP4",
"/home/jovyan/work/output/START_S042_T1_La2/START_S042_T1_La2.MP4",
"/home/jovyan/work/output/START_S045_T1_La1/START_S045_T1_La1.MP4",
"/home/jovyan/work/output/START_S045_T1_La2/START_S045_T1_La2.MP4",
"/home/jovyan/work/output/START_S049_T1_La1/START_S049_T1_La1.MP4",
"/home/jovyan/work/output/START_S049_T1_La2/START_S049_T1_La2.MP4",
]

In [ ]:
destination_folders = [d for d in BASE_PATH.iterdir() if d.is_dir()]
destination_folders 

In [ ]:
def find_destination_folder(video_path):
    session_id = Path(video_path).parts[-2].split('_')[1]
    for folder in destination_folders:
        if folder.name.endswith(session_id):
            return folder
    return None

for video_path in video_paths:
    dest_folder = find_destination_folder(video_path)
    if dest_folder:
        shutil.copy(video_path, dest_folder)
        print(f"Copied {video_path} to {dest_folder}")
    else:
        print(f"No matching folder found for {video_path}")


In [ ]:
BASE_PATH = Path('/home/jovyan/work/videos/cross-label')
VIDEO_PATHS = list(BASE_PATH.rglob('*.MP4')) + list(BASE_PATH.rglob('*.mp4'))

CSV_PATHS = list(BASE_PATH.rglob('*.csv'))
TXT_PATHS = list(BASE_PATH.rglob('*.txt'))


print(len(VIDEO_PATHS))
print(len(TXT_PATHS))

In [ ]:
CSV_PATHS = list(BASE_PATH.rglob('*.csv'))
CSV_PATHS

In [ ]:
ELAN = '_ELAN'
SYNC = '_sync'

idx_category = {
    0.0: 'neutral',
    1.1: 'negative',
    1.2: 'positive'
}
category_idx = {
    'neutral': 0.0,
    'negative': 1.1, 
    'positive': 1.2
}

PATH = 'path'
TIME_FILE = 'start_end_file'
SYNC_FILE = 'sync_file'
START = 'start_time'
END = 'end_time'
MU = 'Mu_df'
KI = 'Ki_df'

In [ ]:
meta_dict = {}

for video_path in tqdm(VIDEO_PATHS):
    name = video_path.stem[:13]
    version = video_path.stem[14:]
    folder = Path(video_path.parent.parent)

    meta_dict[str(name)] = {
        'path': str(video_path),
        'start_end_file': "",
        'sync_file': "",
        'start_time': '00:00:00',
        'end_time': '00:00:00',
        'Mu_df': '',
        'Ki_df': ''
    } 

    destination_folder = Path('/home/jovyan/work/cross-label') / name
    
    # get the time file
    csvs = [str(path) for path in BASE_PATH.rglob('*.csv')]
    match_csv = [file for file in csvs if re.search(name, file) and re.search(version, file)]
    match_csv = match_csv[0] if match_csv else None

    # get the sync file
    txts = [str(path) for path in BASE_PATH.rglob('*.txt')]
    match_txt = [file for file in txts if re.search(name, file) and re.search('sync', file) and re.search(version, file)]
    match_txt = match_txt[0] if match_txt else None

    if match_csv and match_txt: 
        destination_folder.mkdir(parents=True, exist_ok=True)

        shutil.copy(match_csv, str(destination_folder))
        meta_dict[str(name)]['start_end_file'] = str(destination_folder) + '/' + str(Path(match_csv).name)
        shutil.copy(match_txt, str(destination_folder))
        meta_dict[str(name)]['sync_file'] = str(destination_folder) + '/' + str(Path(match_txt).name)
        
    if str(name) in meta_dict and meta_dict[str(name)]['sync_file'] == "":
        del meta_dict[str(name)]

json_file = '/home/jovyan/work/cross-label/meta_file.json'
with open(json_file, 'w') as f:
    json.dump(meta_dict, f, indent=4)
    

In [ ]:
match_txt = [file for file in txts if re.search('START_S005', file) and re.search('La1', file)]
match_txt


In [ ]:
PATH = Path('/home/jovyan/work/cross-label')
mp4s = [str(path) for path in PATH.rglob('*.MP4')]
csvs = [str(path) for path in PATH.rglob('*.csv')]
txts = [str(path) for path in PATH.rglob('*.txt')]

file_dict = {}

for file in tqdm(mp4s):
    name = Path(file).stem
    version = Path(file).stem[14:]
    file_dict[str(name)] = {
        'path': str(file),
        'start_end_file': "",
        'sync_file': "",
        'start_time': '00:00:00',
        'end_time': '00:00:00',
        'Mu_df': '',
        'Ki_df': ''
    }

    # get time file
    match_csv = [file for file in csvs if re.search(name, file)]
    time_file = match_csv[0] if match_csv else None
    file_dict[str(name)]['start_end_file'] = time_file

    # get sync file
    match_txt = [file for file in txts if re.search(name[:13], file) and re.search(version, file)]
    sync_file = match_txt[0] if match_txt else None
    file_dict[str(name)]['sync_file'] = sync_file

json_file = '/home/jovyan/work/cross-label/meta_file.json'
with open(json_file, 'w') as f:
    json.dump(file_dict, f, indent=4)

In [ ]:
for (file, file_info) in tqdm(file_dict.items()):
    time_file = file_dict[str(file)]['start_end_file']
    if time_file:
        df = pd.read_csv(time_file)
        file_dict[str(file)]['start_time'] = f"{df.iloc[0]['hour']}:{df.iloc[0]['minute']}:{df.iloc[0]['milisecond']}"
        file_dict[str(file)]['end_time'] = f"{df.iloc[1]['hour']}:{df.iloc[1]['minute']}:{df.iloc[1]['milisecond']}"
        
json_file = '/home/jovyan/work/cross-label/meta_file.json'
with open(json_file, 'w') as f:
    json.dump(file_dict, f, indent=4)

In [ ]:
for (file, file_info) in tqdm(file_dict.items()):
    sync_file = file_dict[str(file)]['sync_file']
    if sync_file:
        df = pd.read_csv(sync_file, sep="	", header=None)
        
        if(len(df.columns) >= 10):
             df = df.drop(df.columns[-1], axis=1)
        columns = ['category', 'file', 'timestamp_start_long', 'timestamp_start_short', 'timestamp_end_long', 'timestamp_end_short', 'length_long', 'length_short', 'label']
        df.columns = columns  
        
        # create df for both mother and child and save to output
        filtered_df = df[df['category'].str.contains('SE')].reset_index(drop=True)
        mu_df = filtered_df[filtered_df['category'].str.contains('Mu')].reset_index(drop=True)
        ki_df = filtered_df[filtered_df['category'].str.contains('Ki')].reset_index(drop=True)

        output_dir = Path('/home/jovyan/work/cross-label/' + file[:13])
        output_dir.mkdir(parents=True, exist_ok=True)

        mu_path = str(output_dir) + f'/{file}_MU_DF.csv'
        ki_path = str(output_dir) + f'/{file}_KI_DF.csv'

        mu_df.to_csv(mu_path, index=False)
        ki_df.to_csv(ki_path, index=False)

        file_dict[str(file)][MU] = mu_path
        file_dict[str(file)][KI] = ki_path
        
# save json
json_file = '/home/jovyan/work/cross-label/meta_file.json'
with open(json_file, 'w') as f:
    json.dump(file_dict, f, indent=4)

In [ ]:
time_path = '/home/jovyan/work/cross-label/time_dict.json'

for file, file_info in tqdm(file_dict.items()): 
    #print(file)
    with open(time_path) as f:
        time_dict = json.load(f)

    output_dir = Path(f'/home/jovyan/work/cross-label/frames/{file}')
    output_dir.mkdir(parents=True, exist_ok=True)

    start_time = datetime.datetime.strptime(file_dict[file][START], '%H:%M:%f').replace(microsecond=0)
    start_time = datetime.time(start_time.hour, start_time.minute, int(start_time.strftime('%H:%M:%S:%f')[-6:-4]))
    start_time = datetime.datetime.combine(datetime.date(1900, 1, 1), start_time)
    video_time = datetime.datetime.strptime(time_dict[file], '%H:%M:%S')

    if start_time <= video_time:
            diff = 0
    elif not video_time.time() == datetime.datetime.strptime('00:00:00', '%H:%M:%S').time():
        diff = (start_time - video_time).seconds
    else:
        diff = 0

    # set up cv2 things
    cap = cv2.VideoCapture(str(file_dict[file]['path'])) 
    fps, num_frames = int(cap.get(cv2.CAP_PROP_FPS)), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # beginnning of experiment frame index
    start_idx = int(fps*diff)

    ki_df_path = file_dict[file]['Ki_df']
    mu_df_path = file_dict[file]['Mu_df']

    if not mu_df_path == '' and not mu_df_path == '':
        # get the event dfs
        ki_df = pd.read_csv(ki_df_path)
        mu_df = pd.read_csv(mu_df_path)
    
        ki_df_short = ki_df[['timestamp_start_short', 'label']]
        mu_df_short = mu_df[['timestamp_start_short', 'label']]
        
        for index, (ki_row, mu_row) in enumerate(zip(ki_df_short.iterrows(), mu_df_short.iterrows())):
            # access df data
            _, ki_data = ki_row
            _, mu_data = mu_row
            ki_timestamp, ki_label = ki_data['timestamp_start_short'], ki_data['label']
            mu_timestamp, mu_label = mu_data['timestamp_start_short'], mu_data['label']
        
            # getframe of start of experiment
            frame_idx = int(start_idx + int(ki_timestamp * fps))
            if frame_idx >= num_frames:
                break
                
            # set up cv2 stuff
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            _, frame = cap.read()
            res_path = str(output_dir / f'{file}_frame_{index}_timestamp_{ki_timestamp}_MU_{mu_label}_KI_{ki_label}.jpg')
            cv2.imwrite(res_path, frame)


In [ ]:
extract_frames(time_path, meta_path, file_dict)

#### Other computations

In [ ]:
p = Path('/home/jovyan/work/output/')

path_list = list(p.rglob('*.MP4')) + list(p.rglob('*.mp4'))

print(len(path_list))

In [ ]:
durations = []

for vid in path_list:
    cap = cv2.VideoCapture(str(vid))
    fps, num_frames = int(cap.get(cv2.CAP_PROP_FPS)), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = num_frames / fps
    durations.append(duration)

In [ ]:
import numpy as np
avg_duration = np.mean(durations)
std_duration = np.std(durations)
print(avg_duration, std_duration)    
print(avg_duration / 60, std_duration / 60)    

        

#### Label Analysis

In [ ]:
# remove checkpoints
checkpoints = list(Path('/home/jovyan/work').rglob('.ipynb_checkpoints'))

for file in checkpoints:
    if file.is_dir():        
        for sub_file in file.iterdir():
            if sub_file.is_file():
                sub_file.unlink()
            elif sub_file.is_dir():
                sub_file.rmdir()
    file.rmdir()

print('done')

In [ ]:
p = Path('/home/jovyan/work/output/')

csvs = list(p.rglob('*.csv'))
print(len(csvs))

In [ ]:
mu_ki_csvs = [str(file) for file in csvs if re.search('KI|MU', str(file)) and not re.search('checkpoint', str(file))]
len(mu_ki_csvs)

In [ ]:

df = pd.read_csv('/home/jovyan/work/output/START_S048_T1_La1/START_S048_T1_La1.csv', sep=",")
df

In [ ]:
label_dict = {
    'file': [],
    'labels': []
}

for file in tqdm(mu_ki_csvs):
    file_name = Path(file).parent.name
    df = pd.read_csv(file, sep=",")
    labels = list(df['label'].unique())

    label_dict['file'].append(file_name)
    label_dict['labels'].append(labels)
    
label_df = pd.DataFrame(label_df)

In [ ]:
label_dict['labels']

In [ ]:

df = pd.read_csv(mu_ki_csvs[0], sep=",")
df

In [ ]:
list(df['label'].unique())